In [249]:
from utility import request_json
import requests
import rdflib

### Check from Scicrunch

In [4]:
SCICRUNCH_PRODUCTION = 'sckan-scigraph'
SCICRUNCH_API_ENDPOINT = 'https://scicrunch.org/api/1'
SCICRUNCH_SPARC_API = f'{SCICRUNCH_API_ENDPOINT}/{{SCICRUNCH_RELEASE}}'

SCICRUNCH_SPARC_VOCAB = f'{SCICRUNCH_SPARC_API}/vocabulary/id/{{TERM}}.json'
SCICRUNCH_SPARC_API = f'{SCICRUNCH_API_ENDPOINT}/ilx/search/term/{{LABEL}}'

In [68]:
import os
import pandas as pd
from tqdm import tqdm

tqdm.pandas()

params = {
    'api_key': os.environ.get('SCICRUNCH_API_KEY'),
    'limit': 9999,
}

def get_existing_term(term: str):
    data = request_json(SCICRUNCH_SPARC_VOCAB.format(SCICRUNCH_RELEASE=SCICRUNCH_PRODUCTION, TERM=term), params=params)
    if (labels:=data.get('labels')):
        return get_term_from_label(labels[0])
    return pd.NA

def get_term_from_label(label):
    data = request_json(SCICRUNCH_SPARC_API.format(LABEL=label), params=params)
    existing_ids = [eid['curie'] for eid in data.get('data', {}).get('existing_ids', pd.NA)]
    return existing_ids if existing_ids else pd.NA

In [ ]:
### Load nerve list
df = pd.read_csv('nervesWithVagus_annotations.csv')

### Checking based on Term ID

In [ ]:
fma_df = df[df['Term ID'].astype(str).str.startswith('FMA')]

In [ ]:
fma_df['available'] = fma_df['Term ID'].progress_apply(get_existing_term)

In [82]:
fma_df.to_csv('fma_df.csv', index=False)

### Checking those without Term ID

In [69]:
null_df = df[df['Term ID'].isnull()]
null_df['available'] = null_df['Group name'].progress_apply(get_term_from_label)

100%|██████████| 130/130 [01:29<00:00,  1.44it/s]
/var/folders/zz/mk7_vrcn3r3gsbwr585w4blxzyk8xb/T/ipykernel_74345/2597593306.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  null_df['available'] = null_df['Group name'].progress_apply(get_term_from_label)


In [77]:
null_df[null_df['available'].str.len() > 0].shape

(60, 3)

In [83]:
null_df.to_csv('null_df.csv', index=False)

### Check from uberon, using hasDbXref

In [ ]:
### Dowload ### You can also downoad manually and store it in this repo main directory
### You can also downoad manually and store it in this repo main directory

uberon_url = 'https://data.bioontology.org/ontologies/UBERON/submissions/351/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb'
response = requests.get(uberon_url)
response.raise_for_status()
with open('uberon.owl', "wb") as f:
    f.write(response.content)

In [ ]:
## Load uberon to rdflib graph
g_uberon = rdflib.Graph()
g_uberon.parse('uberon.owl', format='xml')

In [193]:
OBOINOWL = rdflib.Namespace("http://www.geneontology.org/formats/oboInOwl#")

def get_hasDbXref(term):
    for s in g_uberon.subjects(predicate=OBOINOWL.hasDbXref, object=rdflib.Literal(term)):
        if (s, rdflib.RDF.type, rdflib.OWL.Class) in g_uberon:
            return str(s).replace('http://purl.obolibrary.org/obo/UBERON_', 'UBERON:')
    return pd.NA

In [194]:
fma_df = fma_df.copy()
fma_df['available'] = fma_df['available'].fillna(fma_df['Term ID'].apply(get_hasDbXref))

### Now check superclass and superbranch
This is not the match but candidate to check

And this take a lot of time, be patient

In [ ]:
### Dowload and fma.owl
### You can also downoad manually and store it in this repo main directory

uberon_url = 'https://data.bioontology.org/ontologies/UBERON/submissions/351/download?apikey=8b5b7825-538d-40e0-9e9e-5ab9274a9aeb'
response = requests.get(uberon_url)
response.raise_for_status()
with open('uberon.owl', "wb") as f:
    f.write(response.content)



In [ ]:
g_fma = rdflib.Graph()
g_fma.parse('fma.owl', format='xml')

<Graph identifier=N6842709cda384c63a8e76c9d00a33d7e (<class 'rdflib.graph.Graph'>)>

In [196]:
FMA = rdflib.Namespace("http://purl.org/sig/ont/fma/fma")
RDFS = rdflib.Namespace("http://www.w3.org/2000/01/rdf-schema#")

def curie(uri):
    if isinstance(uri, rdflib.URIRef):
        uri = str(uri)
        if uri.startswith("http://purl.org/sig/ont/fma/fma"):
            return "FMA:" + uri.rsplit("fma", 1)[-1]
        elif uri.startswith("http://purl.obolibrary.org/obo/UBERON_"):
            return uri.replace('http://purl.obolibrary.org/obo/UBERON_', 'UBERON:')
        return uri
    return uri

def clean_literal(val):
    if isinstance(val, rdflib.Literal):
        if val.datatype and val.datatype.endswith("integer"):
            return int(val)
        elif val.datatype and val.datatype.endswith("float"):
            return float(val)
        return str(val)
    return val

def get_related_terms(term):
    query = f"""
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX FMA: <http://purl.org/sig/ont/fma/fma>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>

    SELECT ?superclass ?label ?rel (COUNT(?mid) AS ?level)
    WHERE {{
      {{
        # Subclass path
        {term} rdfs:subClassOf* ?mid .
        ?mid rdfs:subClassOf* ?superclass .
        BIND("subClassOf" AS ?rel)
      }}
      UNION
      {{
        # branch_of restriction path
        {term} rdfs:subClassOf* ?mid .
        ?mid rdfs:subClassOf* [
          owl:onProperty <http://purl.org/sig/ont/fma/branch> ;
          owl:someValuesFrom ?superclass
        ] .
        BIND("branch_of" AS ?rel)
      }}

      # Shared filters and info
      OPTIONAL {{ ?superclass rdfs:label ?label }}
      ?superclass a owl:Class .
      ?superclass rdfs:subClassOf* FMA:65132 .
    }}
    GROUP BY ?superclass ?label ?type ?rel
    ORDER BY ?level

    """

    rows = []
    for row in g_fma.query(query):
        superclass = curie(row.superclass)
        if superclass == 'FMA:65132':
            continue
        label = clean_literal(row.label)
        rel = clean_literal(row.rel)
        level = clean_literal(row.level)
        rows.append((superclass, label, rel, level))

    # return pd.DataFrame(rows, columns=["superclass", "label", "relation", "level"])
    return rows


In [154]:
### now get superclass or super branch of terms (WARNING: this will be slow)
candidate_terms = {}
for term in tqdm(fma_df[fma_df['available'].isna()]['Term ID']):
    candidate_terms[term] = get_related_terms(term)

100%|██████████| 432/432 [35:43<00:00,  4.96s/it]


In [179]:
### populate existing terms, so can be access several time
existing_terms = {}

# Flatten and deduplicate candidate terms
unique_rows = set(row for k_terms in candidate_terms.values() for row in k_terms)

for row in tqdm(unique_rows, desc="Checking existing terms"):
    ilx_terms = get_existing_term(row[0])
    if isinstance(ilx_terms, list) and len(ilx_terms) > 0:
        existing_terms[row[0]] = ilx_terms

Checking existing terms: 100%|██████████| 1211/1211 [29:58<00:00,  1.49s/it]


In [244]:
### then check from existing_terms
ct_df = pd.DataFrame(columns=['Term ID', 'ILX superclass', 'FMA superclass', 'superclass label', 'relation', 'level'])
missing_fmas = []
for term, k_terms in tqdm(candidate_terms.items()):
    for row in k_terms:
        new_row = []
        if (ilx_terms:=existing_terms.get(row[0])):
            new_row = {
                'Term ID': term,
                'ILX superclass': ilx_terms,
                'FMA superclass': row[0],
                'superclass label': row[1],
                'relation': row[2],
                'level': row[3]
            }
            ct_df = pd.concat([ct_df, pd.DataFrame([new_row])], ignore_index=True)
            break
        if not new_row:
            missing_fmas += [term]

100%|██████████| 429/429 [00:00<00:00, 2725.09it/s]


In [245]:
## Final step, for missing FMA terms check to UBERON

OBOINOWL = rdflib.Namespace("http://www.geneontology.org/formats/oboInOwl#")

for term in missing_fmas:
    for row in candidate_terms[term]:
        for s in g_uberon.subjects(predicate=OBOINOWL.hasDbXref, object=rdflib.Literal(curie)):
            if (s, rdflib.RDF.type, rdflib.OWL.Class) in g_uberon:
                new_row = {
                    'Term ID': term,
                    'ILX superclass': [curie[s]],
                    'FMA superclass': row[0],
                    'superclass label': row[1],
                    'relation': row[2],
                    'level': row[3]
                }
                ct_df = pd.concat([ct_df, pd.DataFrame([new_row])], ignore_index=True)
                break  # break inner 'row' loop
        else:
            continue  # only runs if no break: keep checking rows
        break

### Now combine all

In [255]:
df_merged = df.merge(fma_df[['Term ID', 'available']], on='Term ID', how='left')

df_merged = df_merged.merge(null_df[['Group name', 'available']].rename(columns={'available': 'group_available'}),
                            on='Group name', how='left')

df_merged['available'] = df_merged['available'].fillna(df_merged['group_available'])

df_merged.drop(columns=['group_available'], inplace=True)


final_df = df_merged.merge(
    ct_df,
    on='Term ID',
    how='left'  # or 'outer' if you want to preserve all terms from both
)

In [256]:
final_df.to_csv('mapped_fma_nerves.csv')

In [260]:
final_df[final_df['Group name']=='Chorda tympani branch of left facial nerve']

,Term ID,Group name,available,ILX superclass,FMA superclass,superclass label,relation,level
112,NaN,Chorda tympani branch of left facial nerve,[ILX:0795106],NaN,NaN,NaN,NaN,NaN
